In [1]:
!pip install langchain-groq gradio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 9.5 MB/s eta 0:00:00


In [ ]:


import gradio as gr
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import HumanMessage, AIMessage
from langchain_groq import ChatGroq
from langchain_core.runnables import RunnableSequence

class MedicalChatbot:
    def __init__(self, groq_api_key):
        self.llm = ChatGroq(
            temperature=0.7,
            model_name="gpt-3.5-turbo",
            groq_api_key=groq_api_key
        )

        self.prompt_template = ChatPromptTemplate.from_messages([
            ("system", """You are a helpful medical assistant. Your role is to provide general information about possible
            conditions and recommend safe over-the-counter medications based on user symptoms. Always include disclaimers.

When responding:
- Give 1-2 possible conditions (with disclaimer)
- Recommend OTC medications
- Mention when to seek medical attention
- Give lifestyle advice
- Use friendly tone

Always include:
- "I am not a doctor and this is not medical advice."
- "Consult a healthcare professional before taking any medication."
- "For emergencies, call emergency services immediately."
"""),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{symptoms}")
        ])

        self.chain = self.prompt_template | self.llm
        self.chat_history = []

    def get_advice(self, symptoms):
        try:
            if not symptoms.strip():
                return "Please describe your symptoms."

            self.chat_history.append(HumanMessage(content=symptoms))

            response = self.chain.invoke({
                "chat_history": self.chat_history,
                "symptoms": symptoms
            })

            self.chat_history.append(AIMessage(content=response.content))
            return response.content

        except Exception as e:
            return f"⚠️ Error processing your request: {str(e)}"

# Replace with your actual Groq API key
GROQ_API_KEY = "Your api key"  # <<< PUT YOUR REAL KEY HERE

# Initialize chatbot
chatbot = MedicalChatbot(GROQ_API_KEY)

def respond(message, chat_history):
    if not message.strip():
        return "", chat_history

    bot_response = chatbot.get_advice(message)
    chat_history.append((message, bot_response))
    return "", chat_history

def clear_chat():
    chatbot.chat_history = []
    return []

with gr.Blocks(title="MediChat", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 🩺 MediChat - Medical Information Assistant
    <div style='background-color: #FFF3CD; padding: 10px; border-radius: 5px; margin-bottom: 20px;'>
    <b>⚠️ Disclaimer:</b> This is not medical advice. Consult a doctor for serious symptoms.
    </div>
    """)

    chatbot_ui = gr.Chatbot(
        height=500,
        bubble_full_width=False,
        avatar_images=(
            "https://cdn-icons-png.flaticon.com/512/2784/2784487.png",  # User
            "https://cdn-icons-png.flaticon.com/512/4712/4712035.png"   # Bot
        )
    )

    with gr.Row():
        msg = gr.Textbox(
            label="Describe your symptoms",
            placeholder="I have a headache and fever...",
            scale=4,
            container=False
        )
        submit = gr.Button("Send", variant="primary", scale=1)

    clear = gr.Button("Clear Chat")

    # Event handlers
    msg.submit(respond, [msg, chatbot_ui], [msg, chatbot_ui])
    submit.click(respond, [msg, chatbot_ui], [msg, chatbot_ui])
    clear.click(clear_chat, None, chatbot_ui)

    # Additional info
    with gr.Accordion("Emergency Warning Signs", open=False):
        gr.Markdown("""
        Seek immediate medical care for:
        - 🚨 Chest pain or pressure
        - 🚨 Difficulty breathing
        - 🚨 Severe bleeding
        - 🚨 Sudden weakness/numbness
        - 🚨 Suicidal thoughts
        - 🚨 Seizures or unconsciousness
        """)

# Launch with try-catch for Colab compatibility
try:
    demo.launch(share=True, debug=True)
except Exception as e:
    print(f"Error launching interface: {str(e)}")
    demo.launch(debug=True)

/tmp/ipython-input-2965681941.py:82: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_ui = gr.Chatbot(
/tmp/ipython-input-2965681941.py:82: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot_ui = gr.Chatbot(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://66e3e5f0ea1a2b0054.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
